# Evaluation of the MQT Predictor Performance (compatible with `mqt.predictor==2.0.0`)

In [ ]:
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit

from mqt.predictor import ml, rl

df = pd.read_csv(ml.helper.get_path_results(), sep=",")

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 18


mqt_color = "#1E88E5"
best_color = "#004D40"
median_color = "#FFC107"
worst_color = "#D81B60"
marker_mqt = "+"
marker_best = "o"
marker_median = "."
marker_worst = "x"

# 2x2 Matrix With Mean Results and Optimization Criterion Comparison

In [ ]:
tmp_df = df[df["MQTPredictor_expected_fidelity_expected_fidelity"] >= 0]
tmp_df = tmp_df[tmp_df["MQTPredictor_critical_depth_expected_fidelity"] >= 0]
MQT_expected_fidelity = [
    tmp_df["MQTPredictor_expected_fidelity_expected_fidelity"].mean(),
    tmp_df["MQTPredictor_expected_fidelity_critical_depth"].mean(),
]
MQT_critical_depth = [
    tmp_df["MQTPredictor_critical_depth_expected_fidelity"].mean(),
    tmp_df["MQTPredictor_critical_depth_critical_depth"].mean(),
]

In [ ]:
print(np.round(MQT_expected_fidelity, 2))
print(np.round(MQT_critical_depth, 2))

## Calculate Data Points

In [ ]:
def calculate_median(row: pd.core.series.Series, column_name: str) -> float:
    columns = [col for col in row.index if col.endswith(column_name) and "MQT" not in col]
    values = [value for col, value in row[columns].items() if value != -1]
    return np.median(values) if values else None


df["expected_fidelity_median"] = df.apply(calculate_median, column_name="_expected_fidelity", axis=1)
df["critical_depth_median"] = df.apply(calculate_median, column_name="_critical_depth", axis=1)
df["expected_fidelity_min_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_expected_fidelity") and "MQT" not in col]]
    .replace(-1, np.nan)
    .min(skipna=True),
    axis=1,
)
df["expected_fidelity_max_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_expected_fidelity") and "MQT" not in col]].max(), axis=1
)
df["critical_depth_min_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_critical_depth") and "MQT" not in col]]
    .replace(-1, np.nan)
    .min(skipna=True),
    axis=1,
)
df["critical_depth_max_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_critical_depth") and "MQT" not in col]].max(), axis=1
)

# Expected Fidelity

In [ ]:
predictor = ml.Predictor()

training_data = predictor.get_prepared_training_data(figure_of_merit="expected_fidelity", save_non_zero_indices=True)
indices_test = training_data.indices_test
names_list = training_data.names_list

test_benchmarks_expected_fidelity = [names_list[index_test] for index_test in indices_test]
df_filtered_expected_fidelity = df[df["file_path"].isin(test_benchmarks_expected_fidelity)]
df_filtered_expected_fidelity = df_filtered_expected_fidelity[
    df_filtered_expected_fidelity["MQTPredictor_expected_fidelity_expected_fidelity"] >= 0
]

In [ ]:
df_filtered_expected_fidelity["MQT_Predictor_Improvement_expected_fidelity"] = (
    df_filtered_expected_fidelity["MQTPredictor_expected_fidelity_expected_fidelity"]
    / df_filtered_expected_fidelity["expected_fidelity_max_other"]
)
df_filtered_expected_fidelity.sort_values("MQT_Predictor_Improvement_expected_fidelity")

In [ ]:
plot_kind = "scatter"
scatter_plot_size = 80
df_filtered_and_sorted_expected_fidelity = df_filtered_expected_fidelity.sort_values(
    by=["MQTPredictor_expected_fidelity_expected_fidelity"]
)
df_filtered_and_sorted_expected_fidelity = df_filtered_and_sorted_expected_fidelity[
    ~(
        (df_filtered_and_sorted_expected_fidelity["expected_fidelity_max_other"] == 0)
        & (df_filtered_and_sorted_expected_fidelity["MQTPredictor_expected_fidelity_expected_fidelity"] == 0)
    )
]

ax = df_filtered_and_sorted_expected_fidelity.plot(
    x="file_path",
    y="expected_fidelity_min_other",
    label="Worst",
    rot=90,
    kind=plot_kind,
    color=worst_color,
    marker=marker_worst,
    s=scatter_plot_size,
)
df_filtered_and_sorted_expected_fidelity.plot(
    x="file_path",
    y="expected_fidelity_median",
    kind=plot_kind,
    rot=90,
    ax=ax,
    color=median_color,
    label="Median",
    marker=marker_median,
    s=scatter_plot_size,
)
df_filtered_and_sorted_expected_fidelity.plot(
    x="file_path",
    y="expected_fidelity_max_other",
    label="Best",
    rot=90,
    ax=ax,
    kind=plot_kind,
    color=best_color,
    marker=marker_best,
    s=scatter_plot_size,
    figsize=[16, 6.0],
)
df_filtered_and_sorted_expected_fidelity.plot(
    x="file_path",
    y="MQTPredictor_expected_fidelity_expected_fidelity",
    label="MQT Predictor",
    kind=plot_kind,
    rot=90,
    ax=ax,
    color=mqt_color,
    marker=marker_mqt,
    s=scatter_plot_size,
)


if plot_kind == "line":
    plt.xticks(
        range(len(df_filtered_and_sorted_expected_fidelity.file_path)),
        df_filtered_and_sorted_expected_fidelity.file_path,
    )

plt.xticks(
    list(range(0, len(df_filtered_and_sorted_expected_fidelity.file_path), 1)),
    ["" for i in range(0, len(df_filtered_and_sorted_expected_fidelity.file_path), 1)],
    fontsize=18,
)

plt.xlabel("Benchmarks")
plt.ylabel("Expected Fidelity")

handles, labels = plt.gca().get_legend_handles_labels()
order = [3, 2, 1, 0]
plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order])

plt.savefig("results/expected_fidelity_plot.pdf", bbox_inches="tight")

## Top 3 expected_fidelity

In [ ]:
df_filtered_and_sorted_expected_fidelity["Rank_MQT_expected_fidelity"] = df_filtered_and_sorted_expected_fidelity.apply(
    lambda row: sum(
        1
        for col in df_filtered_and_sorted_expected_fidelity.columns
        if col.endswith("_expected_fidelity")
        and "MQT" not in col
        and row["MQTPredictor_expected_fidelity_expected_fidelity"] < row[col]
    ),
    axis=1,
)

plt.hist(
    df_filtered_and_sorted_expected_fidelity.Rank_MQT_expected_fidelity.values,
    bins=range(0, 15, 1),
    align="left",
    weights=np.ones(len(df_filtered_and_sorted_expected_fidelity)) / len(df_filtered_and_sorted_expected_fidelity),
)
plt.xticks(range(0, 15, 1));

In [ ]:
count_top3 = 0
for _index, row in df_filtered_and_sorted_expected_fidelity.iterrows():
    if row["Rank_MQT_expected_fidelity"] in [0, 1, 2]:
        count_top3 += 1

print("Percentage of Top-3:", count_top3 / len(df_filtered_and_sorted_expected_fidelity))

# Critical Depth

In [ ]:
predictor = ml.Predictor()
training_data = predictor.get_prepared_training_data(figure_of_merit="critical_depth", save_non_zero_indices=True)
indices_test = training_data.indices_test
names_list = training_data.names_list

test_benchmarks_critical_depth = [names_list[index_test] for index_test in indices_test]
df_filtered_critical_depth = df[df["file_path"].isin(test_benchmarks_critical_depth)]
df_filtered_critical_depth = df_filtered_critical_depth[
    df_filtered_critical_depth["MQTPredictor_critical_depth_critical_depth"] >= 0
]

In [ ]:
df_filtered_critical_depth["MQT_Predictor_Improvement_critical_depth"] = (
    df_filtered_critical_depth["MQTPredictor_critical_depth_critical_depth"]
    / df_filtered_critical_depth["critical_depth_max_other"]
)
df_filtered_critical_depth.sort_values("MQT_Predictor_Improvement_critical_depth")

In [ ]:
plot_kind = "scatter"
df_filtered_and_sorted_critical_depth = df_filtered_critical_depth.sort_values(
    by=["MQTPredictor_critical_depth_critical_depth"]
)
df_filtered_and_sorted_critical_depth = df_filtered_and_sorted_critical_depth[
    ~(
        (df_filtered_and_sorted_critical_depth["critical_depth_max_other"] == 0)
        & (df_filtered_and_sorted_critical_depth["MQTPredictor_critical_depth_critical_depth"] == 0)
    )
]

ax = df_filtered_and_sorted_critical_depth.plot(
    x="file_path",
    y="critical_depth_min_other",
    label="Worst",
    rot=90,
    kind=plot_kind,
    color=worst_color,
    marker=marker_worst,
    s=scatter_plot_size,
)
df_filtered_and_sorted_critical_depth.plot(
    x="file_path",
    y="critical_depth_median",
    kind=plot_kind,
    rot=90,
    ax=ax,
    color=median_color,
    label="Median",
    marker=marker_median,
    s=scatter_plot_size,
)
df_filtered_and_sorted_critical_depth.plot(
    x="file_path",
    y="critical_depth_max_other",
    label="Best",
    rot=90,
    kind=plot_kind,
    ax=ax,
    color=best_color,
    marker=marker_best,
    s=scatter_plot_size,
    figsize=[16, 6.0],
)
df_filtered_and_sorted_critical_depth.plot(
    x="file_path",
    y="MQTPredictor_critical_depth_critical_depth",
    kind=plot_kind,
    rot=90,
    ax=ax,
    color=mqt_color,
    label="MQT Predictor",
    marker=marker_mqt,
    s=scatter_plot_size,
)


if plot_kind == "line":
    plt.xticks(
        range(len(df_filtered_and_sorted_critical_depth.file_path)), df_filtered_and_sorted_critical_depth.file_path
    )


plt.xticks(
    list(range(0, len(df_filtered_and_sorted_critical_depth.file_path), 1)),
    ["" for i in range(0, len(df_filtered_and_sorted_critical_depth), 1)],
    fontsize=18,
)

plt.xlabel("Benchmarks")
plt.ylabel("Critical Depth")

handles, labels = plt.gca().get_legend_handles_labels()
order = [3, 2, 1, 0]
plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order])

plt.savefig("results/critical_depth_plot.pdf", bbox_inches="tight")

## Top 3 Critical Depth

In [ ]:
df_filtered_and_sorted_critical_depth["Rank_MQT_critical_depth"] = df_filtered_and_sorted_critical_depth.apply(
    lambda row: sum(
        1
        for col in df_filtered_and_sorted_critical_depth.columns
        if col.endswith("_critical_depth")
        and "MQT" not in col
        and row["MQTPredictor_critical_depth_critical_depth"] < row[col]
    ),
    axis=1,
)

plt.hist(
    df_filtered_and_sorted_critical_depth.Rank_MQT_critical_depth.values,
    bins=range(0, 15, 1),
    align="left",
    weights=np.ones(len(df_filtered_and_sorted_critical_depth)) / len(df_filtered_and_sorted_critical_depth),
)
plt.xticks(range(0, 15, 1));

In [ ]:
count_top3 = 0
for _index, row in df_filtered_and_sorted_critical_depth.iterrows():
    if row["Rank_MQT_critical_depth"] in [0, 1, 2]:
        count_top3 += 1

# Print the total count
print("Percentage of Top-3:", count_top3 / len(df_filtered_and_sorted_critical_depth))

# GHZ Evaluation

In [ ]:
df = pd.read_csv(ml.helper.get_path_results(ghz_results=True), sep=",")
df = df[df.num_qubits < 32]
plt.rcParams["font.size"] = 18

In [ ]:
df["expected_fidelity_median"] = df.apply(calculate_median, column_name="_expected_fidelity", axis=1)
df["critical_depth_median"] = df.apply(calculate_median, column_name="_critical_depth", axis=1)
df["expected_fidelity_min_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_expected_fidelity") and "MQT" not in col]]
    .replace(-1, np.nan)
    .min(skipna=True),
    axis=1,
)
df["expected_fidelity_max_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_expected_fidelity") and "MQT" not in col]].max(), axis=1
)
df["critical_depth_min_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_critical_depth") and "MQT" not in col]]
    .replace(-1, np.nan)
    .min(skipna=True),
    axis=1,
)
df["critical_depth_max_other"] = df.apply(
    lambda row: row[[col for col in row.index if col.endswith("_critical_depth") and "MQT" not in col]].max(), axis=1
)

In [ ]:
plot_kind = "scatter"
df = df.sort_values(by=["num_qubits"])
ax = df.plot(
    x="file_path",
    y="expected_fidelity_max_other",
    label="Best",
    rot=90,
    kind=plot_kind,
    color=best_color,
    marker=marker_best,
    s=scatter_plot_size,
)
df.plot(
    x="file_path",
    y="MQTPredictor_expected_fidelity_expected_fidelity",
    label="MQT Predictor",
    kind=plot_kind,
    rot=90,
    ax=ax,
    color=mqt_color,
    marker=marker_mqt,
    s=scatter_plot_size,
)
df.plot(
    x="file_path",
    y="expected_fidelity_median",
    kind=plot_kind,
    rot=90,
    ax=ax,
    color=median_color,
    label="Median",
    marker=marker_median,
    s=scatter_plot_size,
)
df.plot(
    x="file_path",
    y="expected_fidelity_min_other",
    label="Worst",
    rot=90,
    kind=plot_kind,
    ax=ax,
    color=worst_color,
    marker=marker_worst,
    s=scatter_plot_size,
)

if plot_kind == "line":
    plt.xticks(range(len(df.file_path)), df.file_path)

plt.xticks(
    list(range(0, len(df), 1)),
    [df.iloc[i].num_qubits if i % 4 == 1 else "" for i in range(len(df))],
)

plt.xlabel("Number of Qubits")
plt.ylabel("Expected Fidelity")

handles, labels = plt.gca().get_legend_handles_labels()
order = [1, 0, 2, 3]
plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order], loc="center right")


plt.savefig("results/expected_fidelity_ghz_plot.pdf", bbox_inches="tight")

In [ ]:
plot_kind = "scatter"
df = df.sort_values(by=["num_qubits"])
ax = df.plot(
    x="file_path",
    y="critical_depth_max_other",
    label="Best",
    rot=90,
    kind=plot_kind,
    color=best_color,
    marker=marker_best,
    s=scatter_plot_size,
)
df.plot(
    x="file_path",
    y="MQTPredictor_critical_depth_critical_depth",
    kind=plot_kind,
    rot=90,
    ax=ax,
    color=mqt_color,
    label="MQT Predictor",
    marker=marker_mqt,
    s=scatter_plot_size,
)
df.plot(
    x="file_path",
    y="critical_depth_median",
    kind=plot_kind,
    rot=90,
    ax=ax,
    color=median_color,
    label="Median",
    marker=marker_median,
    s=scatter_plot_size,
)

if plot_kind == "line":
    plt.xticks(range(len(df.file_path)), df.file_path)

plt.xticks(
    list(range(0, len(df), 1)),
    [df.iloc[i].num_qubits if i % 4 == 1 else "" for i in range(len(df))],
)

plt.xlabel("Number of Qubits")
plt.ylabel("Critical Depth")

handles, labels = plt.gca().get_legend_handles_labels()
order = [1, 0, 2]
plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order], loc="center left")

plt.savefig("results/critical_depth_ghz_plot.pdf", bbox_inches="tight")

# Evaluation Journal Revision

In [ ]:
df = pd.read_csv(ml.helper.get_path_results(), sep=",")
plt.rcParams["font.size"] = 12

device = "ibm_washington"
device_index = rl.helper.get_device_index_of_device(device)
devices = rl.helper.get_devices()
dist_colors = ["#fde725", "#90d743", "#35b779", "#21918c", "#31688e", "#443983", "#440154"]


def format_label(label: str) -> str:
    words = label.split("_")
    return " ".join(word.capitalize() for word in words)

## Data Preparation Expected Fidelity

In [ ]:
training_data_expected_fidelity = predictor.get_prepared_training_data(
    figure_of_merit="expected_fidelity", save_non_zero_indices=True
)
indices_test_expected_fidelity = training_data_expected_fidelity.indices_test
names_list_expected_fidelity = training_data_expected_fidelity.names_list
names_test_expected_fidelity = [
    names_list_expected_fidelity[index_test] for index_test in indices_test_expected_fidelity
]
scores_test_expected_fidelity = [
    training_data_expected_fidelity.scores_list[index_test] for index_test in indices_test_expected_fidelity
]
df_expected_fidelity = df[df["file_path"].isin(names_test_expected_fidelity)]
df_expected_fidelity = df_expected_fidelity[
    df_expected_fidelity["MQTPredictor_expected_fidelity_expected_fidelity"] >= 0
]
df_expected_fidelity["RL_" + device + "_expected_fidelity"] = df_expected_fidelity.apply(
    lambda row: max(
        scores_test_expected_fidelity[names_test_expected_fidelity.index(row["file_path"])][device_index], 0
    ),
    axis=1,
)

In [ ]:
len(df_expected_fidelity)

## Data Preparation Critical Depth

In [ ]:
training_data_critical_depth = predictor.get_prepared_training_data(
    figure_of_merit="critical_depth", save_non_zero_indices=True
)
indices_test_critical_depth = training_data_critical_depth.indices_test
names_list_critical_depth = training_data_critical_depth.names_list
names_test_critical_depth = [names_list_critical_depth[index_test] for index_test in indices_test_critical_depth]
scores_test_critical_depth = [
    training_data_critical_depth.scores_list[index_test] for index_test in indices_test_critical_depth
]
df_critical_depth = df[df["file_path"].isin(names_test_critical_depth)]
df_critical_depth = df_critical_depth[df_critical_depth["MQTPredictor_critical_depth_critical_depth"] >= 0]
df_critical_depth["RL_" + device + "_critical_depth"] = df_critical_depth.apply(
    lambda row: max(scores_test_critical_depth[names_test_critical_depth.index(row["file_path"])][device_index], 0),
    axis=1,
)

# Actual Device Distributions

In [ ]:
for fom in ["expected_fidelity", "critical_depth"]:
    if fom == "expected_fidelity":
        df = df_expected_fidelity
        df["best_device"] = df.apply(
            lambda row: devices[
                np.argmax(scores_test_expected_fidelity[names_test_expected_fidelity.index(row["file_path"])])
            ]["name"],
            axis=1,
        )
        colors = [dist_colors[0], dist_colors[6], dist_colors[1]]
    else:
        df = df_critical_depth
        df["best_device"] = df.apply(
            lambda row: devices[
                np.argmax(scores_test_critical_depth[names_test_critical_depth.index(row["file_path"])])
            ]["name"],
            axis=1,
        )
        colors = [dist_colors[rl.helper.get_device_index_of_device(device)] for device in df["best_device"].unique()]
    df["num_qubits_interval"] = pd.cut(
        df["num_qubits"],
        [0, 8, 11, 25, 27, 32, 80, 127],
        labels=["1-8", "9-11", "12-25", "26-27", "28-32", "33-80", "81-127"],
    )

    print(df["best_device"].unique(), colors)
    ax = (
        df.groupby(["num_qubits_interval", "best_device"])
        .size()
        .unstack()
        .fillna(0)
        .plot(kind="bar", stacked=True, color=colors)
    )
    ax.set_xlabel("Number of Qubits")
    ax.set_ylabel("Number of Benchmarks")

    handles, labels = ax.get_legend_handles_labels()
    legend_labels = {
        "ionq_harmony": "Ion Trap 11 Qubits",
        "ionq_aria1": "Ion Trap 25 Qubits",
        "quantinuum_h2": "Ion Trap 32 Qubits",
        "oqc_lucy": "Superconducting 8 Qubits",
        "ibm_montreal": "Superconducting 27 Qubits",
        "rigetti_aspen_m2": "Superconducting 80 Qubits",
        "ibm_washington": "Superconducting 127 Qubits",
    }
    filtered_labels = [label for label in labels if label in legend_labels]
    adjusted_labels = [legend_labels[label] for label in filtered_labels]
    adjusted_handles = [handles[labels.index(label)] for label in filtered_labels]

    def custom_sort(label: str) -> tuple[str, int]:
        numbers = re.findall(r"\d+", label)
        number_value = int(numbers[0]) if numbers else float("inf")
        return (re.sub(r"\d+", "", label), number_value)

    adjusted_handles, adjusted_labels = zip(
        *sorted(zip(adjusted_handles, adjusted_labels, strict=False), key=lambda x: custom_sort(x[1])), strict=False
    )
    ax.legend(handles=adjusted_handles, labels=adjusted_labels, title="")
    ax.figure.savefig(f"results/actual_devices_{fom}.pdf", bbox_inches="tight")

# Eval ML

In [ ]:
for fom in ["expected_fidelity", "critical_depth"]:
    df = df_expected_fidelity if fom == "expected_fidelity" else df_critical_depth

    df = df[
        ~(
            (df[f"MQTPredictor_{fom}_{fom}"] == 0)
            & (df["RL_" + device + f"_{fom}"] == 0)
            & (df["qiskit_" + device + f"_{fom}"] == 0)
            & (df["tket_" + device + f"_{fom}"] == 0)
        )
    ]

    df = df.sort_values(by=[f"MQTPredictor_{fom}_{fom}"])

    plt.scatter(
        range(len(df)),
        df["RL_" + device + f"_{fom}"],
        label="MQT Predictor RL Compiler",
        color=best_color,
        marker=marker_best,
    )
    plt.scatter(
        range(len(df)), df["qiskit_" + device + f"_{fom}"], label="Qiskit", color=median_color, marker=marker_median
    )
    plt.scatter(range(len(df)), df["tket_" + device + f"_{fom}"], label="TKET", color=worst_color, marker=marker_worst)
    plt.scatter(
        range(len(df)), df[f"MQTPredictor_{fom}_{fom}"], label="MQT Predictor", color=mqt_color, marker=marker_mqt
    )

    plt.xticks(
        list(range(0, len(df.file_path), 1)),
        ["" for i in range(0, len(df.file_path), 1)],
        fontsize=12,
    )

    handles, labels = plt.gca().get_legend_handles_labels()
    order = [3, 0, 1, 2]
    plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order], loc="upper left")

    plt.xlabel("Benchmarks")
    plt.ylabel(format_label(fom))
    plt.savefig(fname=f"results/ml_{fom}_comparison.pdf", bbox_inches="tight")
    plt.show()

## Eval RL

In [ ]:
qasm_path = ml.helper.get_path_training_circuits()


for fom in ["expected_fidelity", "critical_depth"]:
    df = df_expected_fidelity if fom == "expected_fidelity" else df_critical_depth
    df = df.sort_values(by=[f"MQTPredictor_{fom}_{fom}"])
    df[f"predicted_device_{fom}"] = df.apply(
        lambda row, fom=fom: ml.helper.predict_device_for_figure_of_merit(
            QuantumCircuit.from_qasm_file(qasm_path / (row["file_path"] + ".qasm")), figure_of_merit=fom
        ),
        axis=1,
    )

    MQT = []
    qiskit = []
    tket = []

    for _i, row in enumerate(df.iterrows()):
        MQT_value = row[1][f"MQTPredictor_{fom}_{fom}"]
        qiskit_value = row[1]["qiskit_" + row[1][f"predicted_device_{fom}"] + f"_{fom}"]
        tket_value = row[1]["tket_" + row[1][f"predicted_device_{fom}"] + f"_{fom}"]

        if MQT_value == 0 and qiskit_value == 0 and tket_value == 0:
            continue

        MQT.append(MQT_value)
        qiskit.append(qiskit_value)
        tket.append(tket_value)

    plt.scatter(range(len(MQT)), MQT, label="MQT Predictor RL Compiler", color=best_color, marker=marker_best)
    plt.scatter(range(len(MQT)), qiskit, label="Qiskit", color=median_color, marker=marker_median)
    plt.scatter(range(len(MQT)), tket, label="TKET", color=worst_color, marker=marker_worst)

    plt.xticks(list(range(0, len(MQT), 1)), ["" for i in range(0, len(MQT), 1)], fontsize=18)

    plt.legend()
    plt.xlabel("Benchmarks")
    plt.ylabel(format_label(fom))
    plt.savefig(fname=f"results/rl_{fom}_comparison.pdf", bbox_inches="tight")
    plt.show()